In [1]:
# import Libs
import isaacgym
from legged_gym.envs import *
from legged_gym.utils import  get_args, task_registry
import torch
import numpy as np
import time
import matplotlib.pyplot as plt

Importing module 'gym_38' (/home/willw/isaacgym/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/willw/isaacgym/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.0.1+cu117
Device count 2
/home/willw/isaacgym/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/willw/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/willw/.cache/torch_extensions/py38_cu117/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


Warp 0.10.1 initialized:
   CUDA Toolkit: 11.5, Driver: 12.2
   Devices:
     "cpu"    | x86_64
     "cuda:0" | NVIDIA GeForce RTX 3090 (sm_86)
     "cuda:1" | NVIDIA GeForce RTX 3090 (sm_86)
   Kernel cache: /home/willw/.cache/warp/0.10.1


In [2]:
# Set Args
import sys
sys.argv = sys.argv[0:1]+["--task=Dancer_leg"]
# sys.argv = sys.argv[0:1]+["--task=wk4","--headless"]
sys.argc = len(sys.argv)
args = get_args()

In [3]:
env_cfg, train_cfg = task_registry.get_cfgs(name=args.task)
env_cfg.terrain.mesh_type = "plane"
env_cfg.domain_rand.randomize_init_dof = False
# env_cfg.env.num_envs = min(env_cfg.env.num_envs, 4)
env_cfg.env.num_envs = 1200

In [4]:
env, _ = task_registry.make_env(name=args.task, args=args, env_cfg=env_cfg)
_ = env.reset()

Setting seed: 1731147166.0448782
Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled
self.num_dof: 16 	self.num_dofs: 16
body_names: ['chest', 'left_arm', 'left_hand', 'left_leg_yaw', 'left_leg_roll', 'left_leg_pitch', 'left_leg_knee', 'left_leg_ankle', 'left_foot', 'right_arm', 'right_hand', 'right_leg_yaw', 'right_leg_roll', 'right_leg_pitch', 'right_leg_knee', 'right_leg_ankle', 'right_foot']
dof_names: ['left_arm_upper', 'left_arm_lower', 'left_hip_yaw', 'left_hip_roll', 'left_hip_pitch', 'left_knee', 'left_ankle_pitch', 'left_ankle_roll', 'right_arm_upper', 'right_arm_lower', 'right_hip_yaw', 'right_hip_roll', 'right_hip_pitch', 'right_knee', 'right_ankle_pitch', 'right_ankle_roll']


/home/willw/anaconda3/envs/isaac/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


feet_names: ['left_foot', 'right_foot']
feet_indices: tensor([ 8, 16], device='cuda:0')
forbidden_link_names: ['chest', 'left_leg_pitch', 'right_leg_pitch']
forbidden_link_indices: tensor([ 0,  5, 13], device='cuda:0')
reward_names: ['ang_Vel', 'bHgt', 'bRot', 'bTwist', 'cotr', 'eFrc', 'eVel', 'jVel', 'lin_Vel', 'smth']


In [5]:
_ = env.reset()

In [6]:
# env.depth_image.shape

In [7]:
# action_np = np.array([[0.1,0.3,-1,1.8,0,0,0,-0.3,0.3,0.6,-0.7,0]]*4,dtype=np.float32)
action_np = np.zeros((4,12),dtype=np.float32)
actions = torch.from_numpy(action_np).to(env.device)
# actions = (actions-env.default_dof_pos[:,env.action_dof_indices]) / env.cfg.control.action_scale

In [8]:
env.torques

tensor([[0., 0., -0.,  ..., 0., 0., -0.],
        [0., 0., -0.,  ..., 0., -0., -0.],
        [0., 0., 0.,  ..., 0., 0., -0.],
        ...,
        [0., 0., 0.,  ..., 0., -0., 0.],
        [0., 0., 0.,  ..., 0., -0., -0.],
        [0., 0., 0.,  ..., 0., -0., -0.]], device='cuda:0')

In [9]:
obs_np = np.zeros((500, env.num_envs ,env.num_obs))
image_np = np.zeros((500, env.num_envs ,env.num_actions))

In [10]:
start_time = time.time()
for i in range(500):
    actions = 0.*torch.ones(env.num_envs, env.num_actions, device=env.device)
#     actions = torch_rand_float(-1, 1, [env.num_envs, env.num_actions], device=env.device)
#     (obs, privileged_obs, obs_history, image, est_dict, obs_future), rewards, dones, infos = env.step(actions)
    (obs, privileged_obs, obs_history, est_dict, obs_future), rewards, dones, infos = env.step(actions)
    obs_np[i] = obs.cpu().numpy()
#     print(i, "obs_h, obs:\n", obs_history[0,[0,1,2,3]])
#     print("privileged_obs:\n", privileged_obs[0,:])
#     print("body_vel:\n", body_vel[0,:])
#     print("obs:", obs[:, -4:])
    print("tau:",  torch.where(torch.isnan(env.torques)))
    # if i%100==0:
    #     print(np.round(time.time()-start_time,2), np.round((i+1)*env.dt,2))
print("Done")


tau: (tensor([  47,   47,   47,   47,   47,   47,   47,   47,   47,   47,   47,   47,
          47,   47,   47,   47,  124,  124,  124,  124,  124,  124,  124,  124,
         124,  124,  124,  124,  124,  124,  124,  124,  169,  169,  169,  169,
         169,  169,  169,  169,  169,  169,  169,  169,  169,  169,  169,  169,
         321,  321,  321,  321,  321,  321,  321,  321,  321,  321,  321,  321,
         321,  321,  321,  321,  360,  360,  360,  360,  360,  360,  360,  360,
         360,  360,  360,  360,  360,  360,  360,  360,  362,  362,  362,  362,
         362,  362,  362,  362,  362,  362,  362,  362,  362,  362,  362,  362,
         394,  394,  394,  394,  394,  394,  394,  394,  394,  394,  394,  394,
         394,  394,  394,  394,  523,  523,  523,  523,  523,  523,  523,  523,
         523,  523,  523,  523,  523,  523,  523,  523,  614,  614,  614,  614,
         614,  614,  614,  614,  614,  614,  614,  614,  614,  614,  614,  614,
         770,  770,  770,  770,  7

KeyboardInterrupt: 

In [11]:
env.terrain.height_field_raw

AttributeError: 'WukongLegbase' object has no attribute 'terrain'

In [12]:
ob, rewards, dones, infos = env.step(actions)
[print(o.shape) for o in ob]

torch.Size([4, 49])
torch.Size([4, 152])
torch.Size([4, 2450])


AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
print((obs[:, 18:30] / env.obs_scales.dof_pos) + env.default_dof_pos[:,env.obs_dof_indices])
print(action_np)

In [ ]:
for i in range(4):
    plt.plot(obs_np[:,0,-4+i])
plt.show()

In [ ]:
env.num_heights

In [ ]:
# plt.plot(tArr, obArr[:,:8:1],label=["1","2","3","4","11","21","31","41"])
# plt.legend()
# plt.show()

In [ ]:
from legged_gym.utils.gait_manager import GaitManager
env_cfg.gait.name="walk"
gm = GaitManager(env_cfg.gait,10, num_legs=2, dt=1e-2)

In [ ]:
gait_signal = np.zeros((200,2*2))
t = 1e-2*np.arange(gait_signal.shape[0])
cmd = np.zeros((10,5))
gm.reset(np.arange(10))
for i in range(gait_signal.shape[0]):
    if i==0:
        cmd[0,2] = 0.4
    if i==122:
        cmd[0,2] = 0.
    
    gm.run(cmd)
    gait_signal[i,:] = gm.get_phase_states()[0,:]
# print(gm.footPhases.shape)
# print(gait_signal)

In [ ]:
%matplotlib auto
%matplotlib auto

In [ ]:
plt.figure()
plt.plot(t,gait_signal[:,0]+0.0,"-", label="0")
plt.plot(t,gait_signal[:,1]+1e-2,"-", label="1")
plt.plot(t,gait_signal[:,2]+2e-2,"-", label="2")
plt.plot(t,gait_signal[:,3]-1e-2,"-", label="3")
plt.grid()
plt.legend()

In [ ]:
plt.figure(1)
plt.plot(gm.footPhases[:,0],gait_signal[:,0],"x", label="0")
plt.plot(gm.footPhases[:,0],gait_signal[:,1],"x", label="1")
plt.grid()
plt.legend()
plt.figure(2)
plt.plot(gm.footPhases[:,1],gait_signal[:,2],"x", label="2")
plt.plot(gm.footPhases[:,1],gait_signal[:,3],"x", label="3")
plt.grid()
plt.legend()

In [ ]:
a = gm.get_frc_penalty_coeff()
# print(a.shape)
print(gm.footPhases.shape)
spd = 1.0-a
plt.figure(1)
# print(gm.footPhases)
# print(a)
# plt.plot(gm.footPhases[:,0:2],a[:,0:2],"x")
plt.plot(gm.footPhases[:,0],a[:,0],"x")
plt.grid()
plt.figure(2)
plt.plot(gm.footPhases[:,0],spd[:,0],"x")
plt.grid()

In [ ]:
plt.figure(2)
plt.plot(gm.footPhases[:,0:2])

In [ ]:
contactTolerance = 0.05
swingRatio = 0.5
condf = [lambda x: x <= contactTolerance,
            lambda x: x > 1 - contactTolerance,
            lambda x: abs(x - swingRatio) <= contactTolerance,
            lambda x: np.logical_and(x <= 1 - contactTolerance,
                                     swingRatio + contactTolerance <= x)
            ]
a = np.array([0.55])
for f in condf:
    res = f(a)
    print(res)

tmp1 = abs(a-swingRatio)
print(tmp1)
print(tmp1 == contactTolerance)

In [ ]:
f = lambda x: x>0.5

In [ ]:
v = f(gm.footPhases)
print(v)

In [ ]:
def plot_all(arr):
    x = np.arange(arr.shape[0])
    for i in range(arr.shape[1]):
        fig = plt.figure()
        plt.title(i)
        plt.scatter(x,arr[:,i],marker="x",alpha=0.5)
        plt.show()

In [ ]:
path = "/home/amax/isaacgym/LeggedGym-317/legged_gym/"
a = np.load(f"{path}/act.npy")
o = np.load(f"{path}/obs.npy")
s = np.load(f"{path}/states.npy")

In [ ]:
plot_all(a)

In [ ]:
plot_all(o)

In [ ]:
plot_all(s)

In [ ]:
phi = np.random.random((4096,4))
print(phi)

In [ ]:
isStance = np.random.random((4096,4))>0.5

In [ ]:
cmd = np.random.random((4096,3))

In [ ]:
z = np.repeat(np.expand_dims(np.abs(np.linalg.norm(cmd, axis=-1)) > 0.7,1), 4, 1)

In [ ]:
z.shape

In [ ]:
print(z)
print(~z)

In [ ]:
z &= isStance

In [ ]:
np.logical_and(z,isStance)

In [ ]:
phi[isStance] = 20

In [ ]:
phi

In [ ]:
phi[np.where(np.logical_and(z,isStance))] = 200

In [ ]:
x = torch.rand(5,2)
print(x.size(),'\n',x)
b = x.repeat_interleave(9,dim=1)
print(b.size(),'\n',b)
print(x)